# College Major Analysis based on economical factors

## The goal of the analysis is to provide a data model that can help students and parents choose a college major and understand how big a financial difference it makes.

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
def load_data(filename):
    """Given a filename of a csv load data into a Pandas dataframe.

        filename - string

        return Pandas dataframe
    """
    return pd.read_csv(filename, encoding='utf-8')

**The dataset files used for this analysis are:**
1. all-ages.csv
2. majors-list.csv
3. women-stem.csv
4. recent-grads.csv
5. grad-students.csv

## DataSet Information
**Some columns used in the dataset are:**

* Rank - Rank by median earnings (the dataset is ordered by this column).
* Major_code - Major code.
* Major - Major description.
* Major_category - Category of major.
* Total - Total number of people with major.
* Sample_size - Sample size (unweighted) of full-time.
* Men - Male graduates.
* Women - Female graduates.
* ShareWomen - Women as share of total.
* Employed - Number employed.
* Median - Median salary of full-time, year-round workers.
* Low_wage_jobs - Number in low-wage service jobs.
* Full_time - Number employed 35 hours or more.
* Part_time - Number employed less than 35 hours.


In [ ]:
all_ages = 'all-ages.csv'
grad_students = 'grad-students.csv'
majors_list = 'majors-list.csv'
women_stem = 'women-stem.csv'
recent_grads = 'recent-grads.csv'

In [ ]:
all_ages_df = load_data(all_ages)
recent_grads_df = load_data(recent_grads)
grad_students_df = load_data(grad_students)
majors_list_df = load_data(majors_list)
women_stem_df = load_data(women_stem)

## Data Cleaning (Removing records with missing values)

In [ ]:
def remove_missing_values(df, df_name):
    original_data_len = len(df)
    print("Before removing the missing data and null values, the dataset '{}' has {} number of records".format(df_name, original_data_len))
    
    #drop records with missing values
    df.dropna(inplace = True)
    
    #Now again let's check the no. of records we have
    cleaned_data_len = len(df)
    print("After removing the missing data and null values, the dataset '{}' has {} number of records\n".format(df_name, cleaned_data_len))

In [ ]:
remove_missing_values(all_ages_df, all_ages)
remove_missing_values(recent_grads_df, recent_grads)
remove_missing_values(grad_students_df, grad_students)
remove_missing_values(majors_list_df, majors_list)
remove_missing_values(women_stem_df, women_stem)

In [ ]:
all_ages_df.head(10)

In [ ]:
rows, cols = all_ages_df.shape

print("The number of rows:", rows)
print("The number of columns:", cols)

In [ ]:
grad_students_df.head(10)

In [ ]:
recent_grads_df.head(10)

In [ ]:
majors_list_df.head(10)

In [ ]:
women_stem_df.head(10)

# Summary statistics:

**recent-grads.csv**

In [ ]:
recent_grads_df.describe()

**majors-list.csv**

In [ ]:
majors_list_df.describe()

**all-ages.csv**

In [ ]:
all_ages_df.describe()

**women-stem.csv**

In [ ]:
women_stem_df.describe()

**grad-students.csv**

In [ ]:
grad_students_df.describe()

# Analysis and Visualization

**grad-students.csv**

In [ ]:
# Plot Correlation matrix
def plot_CorrMatrix(df, csv_name):
    corr = df.corr()
    corr.style.background_gradient(cmap='coolwarm')
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()  # to make sure label appears at the bottom
    plt.title(f'Correlation Matrix for {csv_name}', fontsize=18)
    
    plt.show()

In [ ]:
plot_CorrMatrix(grad_students_df, grad_students)

**recent-grads.csv**

In [ ]:
plot_CorrMatrix(recent_grads_df, recent_grads)

# Majors with median salary above $60,000

In [ ]:
high_median_salary = recent_grads_df[recent_grads_df["Median"] > 60000].sort_values("Median")
high_median_salary.plot(x="Major", y=["P25th", "Median", "P75th"], kind="bar", figsize=(8,8))

plt.show()

### From the above bar chart, it is evident that the Petroleum Engineering major graduates got paid the highest out all other majors with a median salary of over $100,000. 

In [ ]:
recent_grads_df.plot(x="Rank", y=["P25th", "Median", "P75th"], figsize=(8,8))
plt.show()

### From the above plot, we can make the following observations:
1. **Median income decreases as the rank decreases since the rank determines the median income**
2. **Some of the majors in the figure have large gaps between the 25th and 75th percentiles. This shows that graduates  with those degrees may earn notably lower or notably higher than the mdedian income.**
3. **All remaining majors have very small gaps between the 25th and 75th percentiles which means that the salary of the graduates with those degrees is very close to the median income.**

## Creating Scatter plots to analyze the relationships between different columns in the recent_grads dataset

In [ ]:
# Total v/ Employed

# to exclude the tail of the series from analysis
recent_grads_modified=recent_grads_df[recent_grads_df["Total"]< recent_grads_df["Total"].quantile(0.9)]

recent_grads_modified.plot(x='Total', y='Employed', kind='scatter', title="Total v/s Employed (reduced dataset)", figsize=(8,8))
plt.show()

### The above scatter plot between Total v/s Employed shows a strong positive correlation. However, we should not forget that correlation doesn't imply causation.

In [ ]:
#Sample_size v/s Median

recent_grads_df.plot(x='Sample_size', y='Median', kind='scatter', title='Sample_size v/s Median', figsize=(8,8))
plt.show()

### From the above scatter plot, we can see that:
1. **Median salaries of the full time employees varies greatly in the lower tier of the sample size**
2. **We can't see any linear relation between the sample size and the median salary**

### Scatter plot with Median and ShareWomen

In [ ]:
plt.figure(figsize=(10,10))
plt.ylim(0,100000)

sns.regplot(x='ShareWomen', y='Median', data=recent_grads_df, scatter=False)
sns.scatterplot(x='ShareWomen', y='Median', data=recent_grads_df, hue='Major_category',
                size='Sample_size', sizes=(10, 800), alpha=0.5, palette='bright')

plt.show()

### From the above Scatterplot, we can see the following observations:
* Slightly but surely we can see a negative correlation between Median and ShareWomen
* The health sector seems like a clear outlier as it has a high ShareWomen along with a high Median salary.
* Petroleum Engineering despite of a small sample size has the highest Median salary.

# Data Distribution

In [ ]:
columns = ["Sample_size", "Median", "Employed", "Full_time", "ShareWomen", "Unemployment_rate", "Men", "Women"]

figure = plt.figure(figsize=(7,25))
for i in range(0,4):
    ax = figure.add_subplot(4,1,i+1)
    ax.set_xlabel(columns[i])
    ax = recent_grads_df[columns[i]].plot(kind='hist', rot=30)
    
figure = plt.figure(figsize=(7,25))
for i in range(4,8):
    ax = figure.add_subplot(4,1,i-3)
    ax.set_xlabel(columns[i])
    ax = recent_grads_df[columns[i]].plot(kind='hist', rot=30)

### From the above histograms, we can conclude that:
1. **From the sample size histogram, it is clearly visible that the majority of the sample size of full-time workers is notably less than 500**
2. **From the Median salary histogram, we can see that the median salary range of recent graduates lies between \\$30,000- \\$40,000**
3. **From the employed column histogram, we can see that less than 50,000 graduate students were able to find some kind of employment** 
4. **From the Full_time histogram, we can see that number of full time employees is less than 25,000**
5. **From the ShareWomen histogram, we can see that share of the women in the total lies in the range of 0.2%-0.75%**
6. **We can see that the unemployment rate has never exceeded over 17.7% from the unemployment_rate histogram**
7. **The last 2 histograms which are number of men and number of women shows us that the number of men is notably similar to the number of women**

# Bar Plots

## Unemployment rate for top 10 ranking majors based on median salary

In [ ]:
# slicing the list up to 10 will give us top 10 ranked majors
recent_grads_df[:10].plot.bar(x='Major', y='Unemployment_rate', figsize=(8,8))

plt.title("Unemployment_rate based on Major")
plt.ylabel("Unemployment_rate")

plt.show()

### From the above Bar plot, we can see that unemployment rate for the top 10 ranked majors is notably low. However, Nuclear Engineering looks like a slight outlier as it has a significantly higher unemployment rate compared to other top 10 ranked majors based on the median salary.

## Unemployment rate for the 10 least ranked majors based on median salary

In [ ]:
# slicing the list from 163 to the end since there are 173 total majors will give us least 10 ranked majors
recent_grads_df[163:].plot.bar(x='Major', y='Unemployment_rate', figsize=(8,8))

plt.title("Unemployment Rate based on Major")
plt.ylabel("Unemployment_rate")
plt.show()

### From the above Bar plot, we can also see that out of 10 least ranked majors, Clinical Psychology looks has the highest unemployment rate followed by Library Science and Other Foreign Languages.

## ShareWomen(women as share of total) for the top 10 ranked majors based on median salary

In [ ]:
# slicing the list up to 10 will give us the top 10 ranked majors
recent_grads_df[:10].plot.bar(x='Major', y='ShareWomen', figsize=(8,8))

plt.title("ShareWomen based on Major")
plt.ylabel("ShareWomen")
plt.show()

### From the above Bar plot, we can see that the top 10 ranking Majors based on Median Salary are notably dominated by males. It is also evident that 7 out of the 10 top ranked Majors has less than 20% female graduates. Astronomy and Astrophysics has notably high womenshare out all other top 10 ranked majors.

## ShareWomen(women as share of total) for the 10 least ranked majors based on median salary

In [ ]:
# slicing the list from 163 to the end since there are 173 total majors will give us least 10 ranked majors
recent_grads_df[163:].plot.bar(x='Major', y='ShareWomen', figsize=(8,8))

plt.title("ShareWomen by Major")
plt.ylabel("ShareWomen")
plt.show()

### From the above Bar plot, we can clearly see that the 10 least ranked Majors are dominated by Female graduates. There are more than 60% female graduates in the 10 least ranked majors based on the median salary.

# Top 10 Popular Majors

In [ ]:
# ascending is set to False as we want top 10 
top_10_popular =recent_grads_df.sort_values(by="Total",axis=0,ascending=False)

fig, ax = plt.subplots(figsize=(8,8))

# barh is used to to make a horizontal bar plot
ax.barh(top_10_popular["Major"][:10],top_10_popular["Total"][:10] )
# slicing all the list to 10 since we want top 10 majors
ax.set_yticks(top_10_popular["Major"][:10])
ax.set_yticklabels(top_10_popular["Major"][:10])
ax.invert_yaxis()  # to read the labels from top-to-bottom
ax.set_xlabel('Total Students')
ax.set_title('Top 10 Majors')

plt.show()

### Hence, from the above plot we can conclude that Psychology is the most popular major out of all other majors.  Political Science and Government is the last major on the top 10 popular majors as it barely made to the top 10 popular majors list.

# 10 Least Popular Majors

In [ ]:
# ascending is set to true to get the least 10 popular majors
least_10_popular = recent_grads_df.sort_values(by="Total",axis=0,ascending=True)

fig, ax = plt.subplots(figsize=(8,8))

# barh to make a horizontal bar plot
ax.barh(least_10_popular["Major"][:10],least_10_popular["Total"][:10])
ax.set_yticks(least_10_popular["Major"][:10])
ax.set_yticklabels(least_10_popular["Major"][:10])
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Total Students')
ax.set_title('Least 10 popular Majos')
plt.show()

### Hence, from the above plot we can conclude that Military Technologies is the least popular major out of all other majors and Court Reporting is at the bottom of top 10 least popular majors

# 15 Most Popular majors breakdown by gender

In [ ]:
# head is set to 15 to get top 15 majors
graduates_major_grouping = recent_grads_df.groupby('Major').sum().sort_values('Total', ascending=False).head(15).reset_index()

major_gender_df = graduates_major_grouping.sort_values('Total')[['Major', 'Men', 'Women']]
major_gender_df.set_index('Major', inplace=True)
major_gender_df.plot(kind='barh', stacked=True, figsize=(8,8), width=0.6)

plt.xlabel('Total')
plt.title('Top 15 most popular majors with gender breakdown', fontdict={'fontsize': 20})
plt.show()

### From the above plot, we can see that:
* Psychology seems to be the most popular major which is highly skewed towards women
* Business Management & Administration and General Business are split evenly among both men and women
* Computer Science seems to be highly skewed towards men
* Nursing, Elementary Education and General Education are the most skewed towards Women out of all the majors

# Median salary variation in each Major category

In [ ]:
plt.figure(figsize=(8,8))
sns.set_style('whitegrid')

variation = recent_grads_df.loc[:, ['Major_category', 'Median']].groupby('Major_category').median() .sort_values(by='Median', ascending=False)

sns.boxplot(x='Median', y='Major_category', data=recent_grads_df, order=variation.index)
plt.show()

### From the above plot, we can see that:
1. **Out of all the major categories, STEM Majors along with Business majors are the highest paid majors.**
2. **Engineering is at the top of the list and the engineering graduates have a salary which varies between \\$40,000 and just below \\$78,000 with the median salary at just below \\$60,000**
3. **We can also see that most women prefer to go in the Engineering field as it is a less favorable major category by the Women**

# Methods

## 1. Clustering (DBSCAN)

In [ ]:
from sklearn import preprocessing

grad_students_clone = grad_students_df.copy()
grad_students_clone["employment_rate"] = grad_students_clone["Grad_employed"]/(grad_students_clone["Grad_employed"]+grad_students_clone["Grad_unemployed"])

# Label encoding major category values
le = preprocessing.LabelEncoder()
grad_students_clone["Major_category_labelled"] = grad_students_clone[["Major_category"]].apply(le.fit_transform)

In [ ]:
def print_labelled_categories(df):
    dict = {}
    for index, row in df.iterrows():
        if row["Major_category_labelled"] not in dict:
            dict[row["Major_category_labelled"]] = row["Major_category"]
    
    print("Mapping of Major_category\n")
    for key in sorted(dict):
        print("%s: %s" % (key, dict[key]))

print_labelled_categories(grad_students_clone)

In [ ]:
from sklearn.cluster import DBSCAN
# from sklearn.preprocessing import StandardScaler

# Compute DBSCAN
def dbscan(X, eps, min_samples):
    # X = StandardScaler().fit_transform(X)
    db = DBSCAN(eps=eps, min_samples=min_samples)
    db.fit(X)
    labels = db.labels_
    n_clusters_ = len(set(labels))
    y_pred = db.fit_predict(X)
    plt.scatter(X[:,0], X[:,1],c=y_pred, cmap='Paired')
    plt.title("Estimated number of clusters: {}".format(n_clusters_))

In [ ]:
transformed_grad_1 = grad_students_clone[["Grad_median","Major_category_labelled"]].copy()
dbscan(transformed_grad_1.values, 2000, 4)

In [ ]:
transformed_grad_2 = grad_students_clone[["employment_rate","Major_category_labelled"]].copy()
dbscan(transformed_grad_2.values, 0.005, 4)

### From the above two graphs, we can see that:
1. **Most of the majors are likely to pay graduate students between \\$60,000-\\$80,000 except category 1 and 6 (Arts, Education)**
2. **Employment rates are pretty high in category 6, but it is infeasible considering the median pay**
3. **From graph 2 we can decipher that even though certain majors fall under the same category, the employment opprtunities vary vastly (category 1,5,10)**
4. **Category 13 (Physical Sciences) seems to be the safest choice with good pay and employment opportunities. But if one is willing to aim for a high end salary, category 7 (Engineering) is a good choice as it pays better than every other category, but some majors have lower employment than others**

## 2. Random Forest Regression

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
recent_grads_df=pd.read_csv("recent-grads.csv")
print(recent_grads_df.shape)
print(recent_grads_df.columns)

In [ ]:
data_x=recent_grads_df[['Major_category',"Unemployment_rate",'College_jobs', 'Non_college_jobs','Low_wage_jobs']]

In [ ]:
print(data_x.shape)

In [ ]:
#Applying one hot encoding on major category
onenc=OneHotEncoder()
x_enc=onenc.fit_transform(data_x.iloc[:,0:1]).toarray()
x_enc =pd.DataFrame(x_enc)

In [ ]:
print(x_enc.head())

In [ ]:
df2=pd.concat([x_enc,data_x.iloc[:,1:]],axis=1)

In [ ]:
print(df2.head)

In [ ]:
x=df2.iloc[:,:-1]
y=df2.iloc[:,[-1]]

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
#Splitting dataset in training dataset and testing dataset
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.9,random_state=42)
print(xtrain.shape)
print(ytrain.shape)
print(xtrain.head())

In [ ]:
#Creating a Random Forest Regression model and fitting it to the training data
forest=RandomForestRegressor(n_estimators=1400,random_state=42)
forest.fit(xtrain,ytrain)

#Predicting values for test data
ypred=forest.predict(xtest)
testdf=pd.DataFrame()
print("Model training executed successfully")

In [ ]:
print(ytest)
print(ypred)


In [ ]:
%matplotlib inline

from sklearn.datasets import load_diabetes

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import LocallyLinearEmbedding, TSNE, Isomap
from time import time
from matplotlib.ticker import NullFormatter
import umap.umap_ as umap
from functools import reduce

plt.style.use('ggplot')
#To suppress unnecesary future warings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
df_all_ages_cpy = all_ages_df.copy()
df_grad_students_cpy = grad_students_df.copy()
df_women_stem_cpy = women_stem_df.copy()
df_recent_grads_cpy = recent_grads_df.copy()



df_women_stem_cpy=df_women_stem_cpy.rename(columns={"Median":"W-stem_Median","Total":"W-stem_Total","Men": "W-stem_Men",
                        "Women": "W-stem_Women"})
df_recent_grads_cpy=df_recent_grads_cpy.rename(columns={"Total":"Rec_grad_Total","Men": "Rec_grad_Men", "Women": "Rec_grad_Women",
                   "ShareWomen":"Rec_grad_ShareWomen","Employed":"Rec_grad_Employed",
                   "Unemployed":"Rec_grad_Unemployed","Unemployment_rate":"Rec_grad_Unemployment_rate",
                  "Median":"Rec_grad_Median","P25th":"Rec_grad_P25th","P75th":"Rec_grad_P75th"})


dfs=[df_all_ages_cpy,df_grad_students_cpy,df_women_stem_cpy,df_recent_grads_cpy]
df= reduce(lambda left,right: pd.merge(left,right,on=['Major_code','Major','Major_category']), dfs)


df=pd.get_dummies(df,columns=['Major','Major_category','Major_code'])
df.fillna(value=0, inplace=True)
X = df.values #feature matrix

In [ ]:
df.shape
df.head()

# **1) PCA - Principal Component Analysis**

In [ ]:
#create the object
scaler =  StandardScaler()

#calculate the mean and standard deviation
scaler.fit(X)

#transform the values
X_scaled = scaler.transform(X)

In [ ]:
pca = PCA(n_components= .95 ,random_state = 2020)
pca.fit(X_scaled)
X_pca = pca.transform(X_scaled)

In [ ]:
print('Variance of all = ', sum(pca.explained_variance_ratio_*100))

In [ ]:
plt.plot(pca.explained_variance_ratio_)
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance')

In [ ]:
print('Variance for Principal Components = ', np.cumsum(pca.explained_variance_ratio_ * 100)[69])

# **2) SVD - Single Value Decomposition**

In [ ]:
#Fitting the data in SVD class with the features
trun_svd =  TruncatedSVD(n_components=163)
svd = trun_svd.fit_transform(X)

In [ ]:
print('Variance explained by th Principal Components = ', np.cumsum(trun_svd.explained_variance_ratio_ * 100)[1])

In [ ]:
trun_svd =  TruncatedSVD()
A_transformed = trun_svd.fit_transform(df)

In [ ]:
print("Transformed Matrix:")
print(A_transformed)

In [ ]:
print('Variance of all features = ', sum(trun_svd.explained_variance_ratio_*100))

In [ ]:
plt.plot(np.cumsum(trun_svd.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance')

# **3) LLE - Locally Linear Embedding**

In [ ]:
embedding = LocallyLinearEmbedding(n_components = 70)
X_transformed = embedding.fit_transform(X)
X_transformed.shape

In [ ]:
print(X_transformed)

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.figure(figsize = (10, 6))
c_map = plt.cm.get_cmap('jet', 2)
plt.scatter(X_transformed[:, 0], X_transformed[:, 1], s = 15,
            cmap = c_map)
plt.colorbar()
plt.xlabel('LLE-1') , plt.ylabel('LLE-2')
plt.show()